In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

sns.set(style="darkgrid", font_scale=1.2) 
plt.rcParams["font.family"] = "SimHei" 
plt.rcParams["axes.unicode_minus"] = False

# skiprows:读取数据时跳过的行数。 
# 数据的第一行是描述信息，因此使用skiprows跳过。第二行才是标题数据。
# id(贷款人编号)与next_pymnt_d(下一个预定的还款日期)列存在混合类型，显式指定列的类型，这样效率更高。
# Pandas默认以块的形式处理(解析)数据，从而降低内存消耗，但是可能会解析出混合类型。可以使用两种方式来处理:
# 1 显式通过dtype来设置列的类型。
# 2 将low_memory参数的值设置为False(默认为True)。

data = pd.read_csv("Loan.csv", skiprows=1, dtype={"id": np.str, "next_pymnt_d": np.str}) 
# 默认情况下，只显示20列数据。
print(pd.get_option("max_columns"))
# 查看数据集的形状。
print(data.shape)
# 显式指定最大列数。如果为None，指不限制最大显示列数。 
pd.set_option("max_columns", 120) 
data.head()

In [ ]:
# 删除与贷款无关特征。
irrelevant_coumns = [
    "id", "member_id", "funded_amnt", "funded_amnt_inv", "emp_title",
    "issue_d", "url", "desc", "zip_code", "addr_state", "last_credit_pull_d",
    "earliest_cr_line", "addr_state", "title", "last_pymnt_d"
]
data.drop(irrelevant_coumns, axis=1, inplace=True)
# 删除相关性高的特征。
high_relevant = ["grade", "sub_grade"]
data.drop(high_relevant, axis=1, inplace=True)

In [ ]:
data.info()

In [ ]:
# 默认情况下，最多只显示60行数据。 
print(pd.get_option("max_rows"))
# 如果需要显示完整，可以设置最多显示的行数。
# pd.set_option("max_rows", 200)
miss = data.isnull().sum(axis=0)
miss = pd.concat([miss, miss * 100 / data.shape[0]], axis=1) 
miss.columns = ["miss_num", "miss_rate"] 
miss.sort_values("miss_num", inplace=True, ascending=False) 
display(miss)

In [ ]:
ax = sns.barplot(y=miss.index, x=miss["miss_num"])
figsize = (15, miss.shape[0] // 2)
ax.get_figure().set_size_inches(figsize)
for i in range(miss.shape[0]):
    num = miss["miss_num"].iloc[i]
    rate = miss["miss_rate"].iloc[i]
    ax.text(num / 2, i, f"{num}({rate:.2f}%)")

In [ ]:
all_missing = miss[miss["miss_rate"] == 100].index 
print("删除的特征:")
print(all_missing)
data.drop(all_missing, axis=1, inplace=True)

In [ ]:
miss_too_much=miss[(miss["miss_rate"]>80)&(miss["miss_rate"]<100)].index 
print("缺失值过多的特征:")
print(miss_too_much)

In [ ]:
# next_pymnt_d:下一个计划的还款日期。
# mths_since_last_record:自上次记录以来的月数。 
for c in miss_too_much:
    # 1与0只需要1个字节就可以存储。
    data[c] = data[c].isnull().astype(np.int8) 
data.loc[:, miss_too_much].head()

In [ ]:
missing_medium=miss[(miss["miss_rate"]>20)&(miss["miss_rate"]<=80)].index 
print("缺失值适中的特征:")
print(missing_medium)

In [ ]:
from scipy import stats
# 自借款人上次拖欠债务以来的月数。 
sns.distplot(data["mths_since_last_delinq"]) 
stats.normaltest(data["mths_since_last_delinq"].dropna())

In [ ]:
# 建立辅助特征，用来标记特征原有的值是否缺失。
data["mths_since_last_delinq_indicator"] = data["mths_since_last_delinq"].isnull().astype(np.int8) 
m = data["mths_since_last_delinq"].median()
data["mths_since_last_delinq"].fillna(m, inplace=True)

In [ ]:
missing_less=miss[(miss["miss_rate"]>0)&(miss["miss_rate"]<=20)].index 
print("缺失值较少的特征:")
print(missing_less)

In [ ]:
data.dropna(inplace=True)
# 缺失值处理完成后，我们再次来进行检查。 
(data.isnull().sum()>0).any()

In [ ]:
data.duplicated().sum()
#data.drop_duplicates(inplace=True)

In [ ]:
not_number = []
for k, v in data.dtypes.items():
    if not np.issubdtype(v, np.number): 
        not_number.append(k)
print("非数值变量:") 
print(not_number)

In [ ]:
def plot_var(name_list):
    """
    绘制变量的每个类别的数量。
    Parameters 
    ---------- 
    name : str
    变量的名称。
    """
    num = len(name_list)
    row, col = np.ceil(num / 2).astype(np.int32), 2
    fig, ax = plt.subplots(row, col)
    fig.set_size_inches(15, row * 5)
    ax = ax.ravel()
    for index, name in enumerate(name_list):
        v = data[name].value_counts()
        sns.countplot(x=name, data=data, order=v.index, ax=ax[index])
        # 在图像上绘制数值。
        for x, y in enumerate(v):
            t = ax[index].text(x, y, y)
            # 数值居中对齐。
            t.set_ha("center")
        if len(v) > 10:
            ax[index].set_xticklabels(ax[index].get_xticklabels(), rotation=90)


plot_var(["term", "home_ownership", "verification_status", "purpose"])

In [ ]:
columns = ["term", "home_ownership", "verification_status", "purpose"] 
dummy = pd.get_dummies(data[columns])
display(dummy.head())
data = pd.concat([data, dummy], axis=1)
data = data.drop(columns, axis=1)

In [ ]:
# 也可以使用map或者apply方法来实现相同的功能。
data["int_rate"]=data["int_rate"].str.replace("%","").astype(np.float32)
data["revol_util"]=data["revol_util"].str.replace("%","").astype(np.float32)
 

In [ ]:
map_dict = {
    "10+ years": 10,
    "9 years": 9,
    "8 years": 8,
    "7 years": 7,
    "6 years": 6,
    "5 years": 5,
    "4 years": 4,
    "3 years": 3,
    "2 years": 2,
    "1 year": 1,
    "< 1 year": 0
}
data["emp_length"] = data["emp_length"].map(map_dict)

In [ ]:
plot_var(["pymnt_plan","initial_list_status","application_type"])

In [ ]:
data.drop(["pymnt_plan","initial_list_status","application_type"],axis=1,inplace=True)

In [ ]:
data["loan_status"].value_counts()

In [ ]:
def mapping(key):
    if "Fully Paid" in key:
        return 0 
    else:
        return 1
    
# 选择正常还款与延期的记录。
data = data.loc[(data["loan_status"] != "Current") & (data["loan_status"] != "Default")] 
data["loan_status"] = data["loan_status"].map(mapping)
plot_var(["loan_status"])

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split


lr = LogisticRegression(solver="liblinear")
y = data["loan_status"]
X = data.drop("loan_status", axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0) 
lr.fit(X_train, y_train)
y_hat = lr.predict(X_test) 
print("真实值:", y_test.values[:10]) 
print("预测值:", y_hat[:10])

In [ ]:
from sklearn.metrics import confusion_matrix
# 根据传入的真实值与预测值，创建混淆矩阵。
matrix = confusion_matrix(y_true=y_test, y_pred=y_hat) 
print(matrix)

In [ ]:
mat = plt.matshow(matrix, cmap=plt.cm.Blues, alpha=0.5)
label = ["负例", "正例"]
# 获取当前的绘图对象。
ax = plt.gca()
# 可以一次性设置多个属性。
ax.set(xticks=np.arange(matrix.shape[1]),
       yticks=np.arange(matrix.shape[0]),
       xticklabels=label,
       yticklabels=label,
       title="混淆矩阵可视化\n",
       ylabel="真实值",
       xlabel="预测值")
for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
        plt.text(x=j, y=i, s=matrix[i, j], va="center", ha="center")
plt.grid(False)
# Matplotlib 3.1.1版本需要添加如下的代码，否则可视化显示不完整。(bug)
# a, b = ax.get_ylim()
# ax.set_ylim(a + 0.5, b - 0.5) 
# plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("正确率:", accuracy_score(y_test, y_hat))
# 默认将1类别视为正例，可以通过pos_label参数指定。
print("精准率:", precision_score(y_test, y_hat)) 
print("召回率:", recall_score(y_test, y_hat)) 
print("F1调和平均值:", f1_score(y_test, y_hat))
# 我们也可以调用逻辑回归模型对象的score方法，也能获取正确率。
# 但是需要注意，score方法与正确率(accuracy_score)函数的参数是不同的。 
print("score方法计算正确率:", lr.score(X_test, y_test))

In [ ]:
fromsklearn.metricsimportclassification_report

print(classification_report(y_true=y_test,y_pred=y_hat))

In [ ]:
from sklearn.metrics import roc_curve, auc, roc_auc_score

probo = lr.predict_proba(X_test)
# 返回ROC曲线相关值。返回FPR，TPR与阈值。当分值达到阈值时，将样本判定为正类，
# 否则判定为负类。
# y_true:二分类的标签值(真实值)。
# y_score:每个标签(数据)的分值或概率值。当该值达到阈值时，判定为正例，否则判定为负例。
# 在实际模型评估时，该值往往通过决策函数(decision_function)或者概率函数(predict_proba)获得。 
# pos_label:指定正例的标签值。
fpr, tpr, thresholds = roc_curve(y_true=y_test, y_score=probo[:, 1], pos_label=1)
# 对概率降序排列，然后从中选择若干元素作为阈值，每个阈值下，都可以计算一个tpr与fpr，
# 每个tpr与fpr对应ROC曲线上的一个点，将这些点进行连接，就可以绘制ROC曲线。
print(probo.shape, fpr.shape, tpr.shape, thresholds.shape)
print(thresholds[:10])
# auc与roc_auc_score函数都可以返回AUC面积值，但是注意，两个函数的参数是不同的。 
print("AUC面积值:", auc(fpr, tpr))
print("AUC面积得分:", roc_auc_score(y_true=y_test, y_score=probo[:, 1]))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(fpr, tpr, marker="o", label="ROC曲线") 
plt.plot([0,1], [0,1], lw=2, ls="--", label="随机猜测") 
plt.plot([0, 0, 1], [0, 1, 1], lw=2, ls="-.", label="完美预测") 
plt.xlim(-0.01, 1.02)
plt.ylim(-0.01, 1.02)
plt.xticks(np.arange(0, 1.1, 0.1))
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel("False Positive Rate(FPR)")
plt.ylabel("True Positive Rate(TPR)")
plt.grid(True)
plt.title(f"ROC曲线-AUC值为{auc(fpr, tpr):.3f}")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve

# 计算在不同阈值下的精准率与召回率。
# y_true:每个样本的真实值。
# probas_pred:每个样本的概率值(或z值)。
# pos_label:指定正例类别。
# 函数会返回3个值:
# precision:每个样本在对应阈值下的精准率，最后一个元素值为1。
# recall:每个样本在对应阈值下的召回率。最后一个元素值为0。
# thresholds:升序排列的阈值数组。【说明:因为阈值是升序排列的，因此，可以推导出
# 精准率升序排列，召回率降序排列。】
precision, recall, thresholds = precision_recall_curve(y_test, probo[:, 1], pos_label=1) 
# 阈值会从y_true参数的数组中来选取一部分。注意:阈值的数量比precision与recall少1。 
print(precision.shape, recall.shape, thresholds.shape, y_test.shape)
plt.plot(recall, precision)
plt.xlabel("召回率")
plt.ylabel("精准率")
plt.title("P-R曲线")
 

In [ ]:
min_recall = 0.95
# 召回率降序排列，截取的一定是数组中的前半部分。
boundary_index = recall[recall >= min_recall].shape[0] 
pre = precision[:boundary_index]
rec = recall[:boundary_index]
thr = thresholds[:boundary_index]
f1 = (2 * precision * recall) / (precision + recall) 
index = f1.argmax()
print("最佳阈值:", thr[index])
print("最佳F1值:", f1.max()) 
print("最佳F1值时的精准率:", pre[index]) 
print("最佳F1值时的召回率:", rec[index])

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true=y_test, y_score=probo[:, 1], pos_label=1) 
# roc_curve中，thresholds索引为0的元素值会大于1，为了能够正常显示，
# 将thresholds索引为0的元素修改为1。
thresholds[0] = 1
plt.plot(thresholds, tpr, label="TPR") 
plt.plot(thresholds, fpr, label="FPR") 
plt.legend()
plt.xlabel("阈值")
plt.ylabel("数值") 
plt.title("KS曲线")

In [ ]:
diff = tpr - fpr
index = diff.argmax() 
print("最大差值位置:", index) 
print("对应的阈值:", thresholds[index])